In [1]:
import torch
import numpy as np
from torch import nn
from torch.utils.data import DataLoader, Dataset, Subset
import transformers
import json, pickle
from models import *
from tqdm import tqdm
from utils import mapk

/home/b09901073/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class UserCourseDataset(Dataset):
    def __init__(self, users, user_embeds, total_courses=732):
        self.users = users
        self.user_embeds = [torch.from_numpy(user_embed) for user_embed in user_embeds]
        self.total_courses = total_courses

    def __len__(self):
        return len(self.users)

    def __getitem__(self, idx):
        labels = torch.zeros(self.total_courses)
        labels = labels.scatter_(0, torch.tensor(self.users[idx]["labels"]), 1)
        user_embed = self.user_embeds[idx]
        return user_embed, labels

def collate_fn(batch):
    ### pad the user_embeds to the same length
    # print(batch[0][0].shape)
    max_len = max([item[0].shape[0] for item in batch])
    embed_dim = batch[0][0].shape[1]
    user_embeds = torch.stack([torch.cat([item[0], torch.zeros(max_len - item[0].shape[0], embed_dim)], dim=0) for item in batch])
    labels = torch.stack([item[1] for item in batch])
    return user_embeds, labels

In [3]:
user_embed_path = "processed_datas/train_users_embeddings.pkl"
user_path = "processed_datas/train_users.json"
course_embed_path = "processed_datas/train_courses_embeddings.pkl"

with open(user_path, "r") as f:
    users = json.load(f)

with open(user_embed_path, "rb") as f:
    user_embeds = pickle.load(f)

#user_embeds = [user_embed.reshape(1, -1).astype(np.float32) for user_embed in user_embeds]

trainset = UserCourseDataset(users, user_embeds)

user_embed_path = "processed_datas/val_unseen_embeddings.pkl"
user_path = "processed_datas/val_unseen.json"

with open(user_path, "r") as f:
    users = json.load(f)

with open(user_embed_path, "rb") as f:
    user_embeds = pickle.load(f)

#user_embeds = [user_embed.reshape(1, -1).astype(np.float32) for user_embed in user_embeds]


validset = UserCourseDataset(users, user_embeds)

In [4]:
train_labels = [trainset[i][1] for i in range(len(trainset))]
train_labels_count = np.sum(train_labels, axis=0)
json.dump(np.array(train_labels_count).astype(np.int).tolist(), open("processed_datas/train_labels_count.json", "w"))
negative_weight = train_labels_count / len(trainset)

/home/b09901073/anaconda3/envs/adl/lib/python3.8/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/b09901073/anaconda3/envs/adl/lib/python3.8/site-packages/numpy/core/fromnumeric.py:86: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return ufunc.reduce(obj, axis, dt

In [5]:
# def loss_fn(logits, labels, rec_num=50, rec_weight=0.001):
#     positive_mask = labels.type(torch.bool)
#     positive_loss = nn.BCELoss()(logits[positive_mask], labels[positive_mask])
#     rec_loss = nn.L1Loss()(logits.sum(dim=-1), rec_num*torch.ones(logits.shape[0]))
#     return positive_loss + rec_weight * rec_loss

def loss_fn(logits, labels):
    return nn.BCEWithLogitsLoss(pos_weight=200 * torch.ones(732))(logits, labels)


def eval_metric(logits, labels, k=50):
    preds = torch.argsort(logits, dim=-1, descending=True)[:, :k]
    labels = torch.where(labels == 1)
    labels = [labels[1][labels[0] == i].tolist() for i in range(preds.shape[0])]
    return mapk(labels, preds, 50)

In [6]:
train_loader = DataLoader(trainset, batch_size=32, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(validset, batch_size=32, shuffle=True, collate_fn=collate_fn)
model = ValModelv1(nhead=8).cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [7]:
best_topk = 0

for epoch in range(100):
    losses = []
    recs = []
    topks = []
    for user_embeds, labels in tqdm(train_loader):
        user_embeds = user_embeds.cuda()
        logits = model(user_embeds).cpu()
        loss = loss_fn(logits, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        losses.append(loss.item())
        recs.append(torch.sigmoid(logits).sum(dim=-1).mean().item())
        topks.append(eval_metric(logits, labels))
    print("train epoch: {}, loss: {}, rec: {}, topk: {}".format(epoch, np.mean(losses), np.mean(recs), np.mean(topks)))

    losses = []
    recs = []
    topks = []
    model.eval()
    with torch.no_grad():
        for user_embeds, labels in valid_loader:
            user_embeds = user_embeds.cuda()
            logits = model(user_embeds).cpu()
            loss = loss_fn(logits, labels)
            losses.append(loss.item())
            recs.append(torch.sigmoid(logits).sum(dim=-1).mean().item())
            topks.append(eval_metric(logits, labels))
    print("valid epoch: {}, loss: {}, rec: {}, topk: {}".format(epoch, np.mean(losses), np.mean(recs), np.mean(topks)))
    if np.mean(topks) > best_topk:
        best_topk = np.mean(topks)
        torch.save(model.state_dict(), "val_modelv1.pth")
    model.train()

100%|██████████| 1867/1867 [01:12<00:00, 25.69it/s]


train epoch: 0, loss: 0.4033388276916171, rec: 330.6822677064721, topk: 0.030925384256908207
valid epoch: 0, loss: 0.32791201246308754, rec: 275.1023646763393, topk: 0.029817128165930153


100%|██████████| 1867/1867 [01:11<00:00, 26.01it/s]


train epoch: 1, loss: 0.37953317810267, rec: 296.19407363670933, topk: 0.04562856524530621
valid epoch: 1, loss: 0.31842441586675224, rec: 256.8728516966432, topk: 0.03935474293571417


100%|██████████| 1867/1867 [01:11<00:00, 26.02it/s]


train epoch: 2, loss: 0.37428991996992955, rec: 289.38067498638725, topk: 0.04884069129999926
valid epoch: 2, loss: 0.32483375506414164, rec: 293.71946909139444, topk: 0.03609003507125146


  6%|▌         | 112/1867 [00:04<01:08, 25.47it/s]


KeyboardInterrupt: 

In [7]:
train_loader = DataLoader(trainset, batch_size=32, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(validset, batch_size=32, shuffle=True, collate_fn=collate_fn)
model = ValModelv0(nhead=8, in_size=512, dropout=0.1, hidden_size=256).cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

In [8]:
best_score = 0


for epoch in range(30):
    losses = []
    recs = []
    topks = []
    for user_embeds, labels in tqdm(train_loader):
        user_embeds = user_embeds.cuda()
        logits = model(user_embeds).cpu()
        loss = loss_fn(logits, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        losses.append(loss.item())
        recs.append(torch.sigmoid(logits).sum(dim=-1).mean().item())
        topks.append(eval_metric(logits, labels))
    print("train epoch: {}, loss: {}, rec: {}, topk: {}".format(epoch, np.mean(losses), np.mean(recs), np.mean(topks)))

    losses = []
    recs = []
    topks = []
    model.eval()
    with torch.no_grad():
        for user_embeds, labels in valid_loader:
            user_embeds = user_embeds.cuda()
            logits = model(user_embeds).cpu()
            loss = loss_fn(logits, labels)
            losses.append(loss.item())
            recs.append(torch.sigmoid(logits).sum(dim=-1).mean().item())
            topks.append(eval_metric(logits, labels))
    print("valid epoch: {}, loss: {}, rec: {}, topk: {}".format(epoch, np.mean(losses), np.mean(recs), np.mean(topks)))
    if np.mean(topks) > best_score:
        best_score = np.mean(topks)
        torch.save(model.state_dict(), "val_modelv0.pth")
    model.train()

100%|██████████| 1867/1867 [00:58<00:00, 32.12it/s]


train epoch: 0, loss: 0.6755380030934927, rec: 164.993839813543, topk: 0.16413441316519223
valid epoch: 0, loss: 1.276698078264247, rec: 128.60163414085304, topk: 0.0766969926483974


100%|██████████| 1867/1867 [00:57<00:00, 32.46it/s]


train epoch: 1, loss: 0.6028936509148731, rec: 142.6512653532677, topk: 0.19043870196842944
valid epoch: 1, loss: 1.4861792299773666, rec: 109.47398064162705, topk: 0.0821112780426354


100%|██████████| 1867/1867 [00:57<00:00, 32.52it/s]


...

train epoch: 27, loss: 0.3976326773175596, rec: 91.75496312750809, topk: 0.21942767735660557
valid epoch: 27, loss: 3.0455690036108205, rec: 91.69885863838616, topk: 0.07947509644261551


100%|██████████| 1867/1867 [00:58<00:00, 31.68it/s]


train epoch: 28, loss: 0.39198306962712026, rec: 90.35867240918022, topk: 0.2168464699167425
valid epoch: 28, loss: 2.927065229022896, rec: 96.01369193360046, topk: 0.08294846985021492


100%|██████████| 1867/1867 [00:57<00:00, 32.60it/s]


train epoch: 29, loss: 0.38577445560939566, rec: 88.72915635520147, topk: 0.21695910393882917
valid epoch: 29, loss: 2.9405050595383067, rec: 97.51323400224958, topk: 0.08022687792457599


In [17]:
train_loader = DataLoader(trainset, batch_size=512, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(validset, batch_size=512, shuffle=True, collate_fn=collate_fn)

model = ValModelv2().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

def loss_fn(logits, labels, margin=0.1, negative_weight=0.3):
    positive_mask = labels.type(torch.bool)
    positive_loss = torch.clamp(1.0 - logits[positive_mask]-margin, min=0).mean()
    negative_loss = logits[~positive_mask].mean()
    return positive_loss + negative_weight * negative_loss

In [18]:
for epoch in range(10):
    losses = []
    recs = []
    topks = []
    for user_embeds, labels in tqdm(train_loader):
        user_embeds = user_embeds.cuda()
        logits = model(user_embeds).cpu()
        loss = loss_fn(logits, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        losses.append(loss.item())
        recs.append(logits.sum(dim=-1).mean().item())
        topks.append(eval_metric(logits, labels))
    print("train epoch: {}, loss: {}, rec: {}, topk: {}".format(epoch, np.mean(losses), np.mean(recs), np.mean(topks)))

    losses = []
    recs = []
    topks = []
    model.eval()
    with torch.no_grad():
        for user_embeds, labels in valid_loader:
            user_embeds = user_embeds.cuda()
            logits = model(user_embeds).cpu()
            loss = loss_fn(logits, labels)
            losses.append(loss.item())
            recs.append(logits.sum(dim=-1).mean().item())
            topks.append(eval_metric(logits, labels))
    print("valid epoch: {}, loss: {}, rec: {}, topk: {}".format(epoch, np.mean(losses), np.mean(recs), np.mean(topks)))
    model.train()

100%|██████████| 117/117 [00:44<00:00,  2.60it/s]


train epoch: 0, loss: 0.766678167714013, rec: 4.449799065151785, topk: 0.14409379729010702
valid epoch: 0, loss: 0.8116465029509171, rec: 2.4035574975221055, topk: 0.07683248175912097


100%|██████████| 117/117 [00:45<00:00,  2.59it/s]


train epoch: 1, loss: 0.7304033927428417, rec: 2.534630176348564, topk: 0.1774733906402815
valid epoch: 1, loss: 0.7995522151822629, rec: 1.314250567685003, topk: 0.08595529080869732


100%|██████████| 117/117 [00:45<00:00,  2.59it/s]


...

train epoch: 7, loss: 0.5917456557608058, rec: -11.967124009743715, topk: 0.1919395911850983
valid epoch: 7, loss: 0.7585260842157446, rec: -12.235948977263078, topk: 0.08691540319209114


100%|██████████| 117/117 [00:45<00:00,  2.59it/s]


train epoch: 8, loss: 0.569560245061532, rec: -14.517706903636965, topk: 0.19170585932846912
valid epoch: 8, loss: 0.754508508288342, rec: -14.345203524050506, topk: 0.08492243145754241


100%|██████████| 117/117 [00:45<00:00,  2.57it/s]


train epoch: 9, loss: 0.5479295992443705, rec: -17.033421760950333, topk: 0.1903384165568237
valid epoch: 9, loss: 0.7471263201340385, rec: -16.769900695137355, topk: 0.08650586522202565


In [7]:
train_loader = DataLoader(trainset, batch_size=32, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(validset, batch_size=32, shuffle=True, collate_fn=collate_fn)

model = ValModelv3().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

def loss_fn(logits, labels, margin=0.1, negative_weight=0.3):
    positive_mask = labels.type(torch.bool)
    positive_loss = torch.clamp(1.0 - logits[positive_mask]-margin, min=0).mean()
    negative_loss = logits[~positive_mask].mean()
    return positive_loss + negative_weight * negative_loss

/home/b09901073/adlfinal/adl-final/utils.py:26: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  return torch.tensor(dataset)


In [8]:
for epoch in range(100):
    losses = []
    recs = []
    topks = []
    for user_embeds, labels in tqdm(train_loader):
        user_embeds = user_embeds.cuda()
        logits = model(user_embeds).cpu()
        loss = loss_fn(logits, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        losses.append(loss.item())
        recs.append(logits.sum(dim=-1).mean().item())
        topks.append(eval_metric(logits, labels))
    print("train epoch: {}, loss: {}, rec: {}, topk: {}".format(epoch, np.mean(losses), np.mean(recs), np.mean(topks)))

    losses = []
    recs = []
    topks = []
    model.eval()
    with torch.no_grad():
        for user_embeds, labels in valid_loader:
            user_embeds = user_embeds.cuda()
            logits = model(user_embeds).cpu()
            loss = loss_fn(logits, labels)
            losses.append(loss.item())
            recs.append(logits.sum(dim=-1).mean().item())
            topks.append(eval_metric(logits, labels))
    print("valid epoch: {}, loss: {}, rec: {}, topk: {}".format(epoch, np.mean(losses), np.mean(recs), np.mean(topks)))
    model.train()

100%|██████████| 1867/1867 [03:11<00:00,  9.74it/s]


train epoch: 0, loss: 0.203832466253062, rec: 173.9995898638553, topk: 0.015023128256834932
valid epoch: 0, loss: 0.5313135508325074, rec: 88.25882869762378, topk: 0.01273372787846611


100%|██████████| 1867/1867 [03:10<00:00,  9.78it/s]


train epoch: 1, loss: 0.19324947941906642, rec: 102.03775221789401, topk: 0.015853206457859068
valid epoch: 1, loss: 0.8500865481712006, rec: -21.5492885021063, topk: 0.0053002652279394


  5%|▍         | 93/1867 [00:09<03:01,  9.80it/s]


KeyboardInterrupt: 

In [7]:
train_loader = DataLoader(trainset, batch_size=512, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(validset, batch_size=512, shuffle=True, collate_fn=collate_fn)

model = ValModelv4().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

In [8]:
for epoch in range(30):
    losses = []
    recs = []
    topks = []
    for user_embeds, labels in tqdm(train_loader):
        user_embeds = user_embeds.cuda()
        logits = model(user_embeds).cpu()
        loss = loss_fn(logits, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        losses.append(loss.item())
        recs.append(torch.sigmoid(logits).sum(dim=-1).mean().item())
        topks.append(eval_metric(logits, labels))
    print("train epoch: {}, loss: {}, rec: {}, topk: {}".format(epoch, np.mean(losses), np.mean(recs), np.mean(topks)))

    losses = []
    recs = []
    topks = []
    model.eval()
    with torch.no_grad():
        for user_embeds, labels in valid_loader:
            user_embeds = user_embeds.cuda()
            logits = model(user_embeds).cpu()
            loss = loss_fn(logits, labels)
            losses.append(loss.item())
            recs.append(torch.sigmoid(logits).sum(dim=-1).mean().item())
            topks.append(eval_metric(logits, labels))
    print("valid epoch: {}, loss: {}, rec: {}, topk: {}".format(epoch, np.mean(losses), np.mean(recs), np.mean(topks)))
    model.train()

 50%|████▉     | 58/117 [00:32<00:33,  1.78it/s]


KeyboardInterrupt: 